# TeleMetrum

The [TeleMetrum](http://altusmetrum.org/TeleMetrum/) is a recording dual-deploy altimeter for high power model rocketry with integrated GPS and telemetry link.

We use this as our primary recovery device. It has a GPS tracker and a single-axis acceleromer and a pressure sensor.

We can compare the accelerometer in the TeleMetrum to the ADIS in the primary flight computer. They have excellent agreement.

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, argmax, add, average
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

g_0 = 9.80665

columns = loadtxt("../TeleMetrum/2015-07-19-serial-1372-flight-0002.csv",
                  delimiter=',', comments='#',unpack=True,
                  usecols=(4,5,6,10,11,13,16,21,22,25,39,40,41,42,43,44,45,46,47,48,49,
                           50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70))

tele_time = columns[0]
clock = columns[1]
rssi = columns[2]
tele_accel = columns[3]
tele_pressure = columns[4]
tele_alt = columns[5]
tele_temp = columns[6]
tele_gpslock, tele_gpsnsat = columns[7:9]
tele_gpsalt = columns[9]
sat1,sat2,sat3,sat4,sat5,sat6,sat7,sat8,sat9,sat10,sat11,sat12,sat13,sat14,sat15,sat16,sat17,sat18,sat19,sat20 = columns[10:30]
sat21,sat22,sat23,sat24,sat25,sat26,sat27,sat28,sat29,sat30,sat31,sat32 = columns[30:]

columns = loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
acc_x, acc_y, acc_z = columns[6:9]

t_0 = 117853569585227

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)

# lightly lowpass
from scipy.signal import butter, lfilter, freqz

# Filter requirements.
order = 6
fs = 819.2       # sample rate, Hz
cutoff = 20   # desired cutoff frequency of the filter, Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq

# Get the filter coefficients so we can check its frequency response.
b, a = butter(order, normal_cutoff, btype='low', analog=False)
acc_x_filter = lfilter(b, a, acc_x)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"TeleMetrum Accelerometer")
plt.ylabel(r"Acceleration [m/s${}^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(timestamp[200:-200], subtract(acc_x_filter[200:-200], g_0), 'k-', alpha=0.3, lw=2.0, label="ADIS IMU Vertical Accel")
plt.plot(tele_time, tele_accel, alpha=0.75, label="TeleMetrum Vertical Acceleration")

plt.xlim([-5,42])
plt.ylim([-50,130])
ax1.legend(loc=1)
plt.show()

## Pressure

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"TeleMetrum Pressure")
plt.ylabel(r"Pressure [kPa]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(tele_time, divide(tele_pressure, 1000.0), alpha=0.75, label="TeleMetrum Pressure")

plt.xlim([-5,42])
#plt.ylim([-50,130])
#ax1.legend(loc=1)
plt.show()

## Temperature

The temperature seems to spike near peak velocity

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"TeleMetrum Temperature")
plt.ylabel(r"Temperature [${}^0$C]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(tele_time, tele_temp, alpha=0.75, label="TeleMetrum Temperature")

plt.xlim([-5,42])
#plt.ylim([-50,130])
#ax1.legend(loc=1)
plt.show()

## Altitude

In [ ]:
print """The maximum altitude recorded by the pressure sensor on the Telemetrum was %0.0f meters.
""" % (max(tele_alt))

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"TeleMetrum Pressure Altitude AGL")
plt.ylabel(r"Altitude AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(tele_time, divide(tele_alt, 1000.0), alpha=0.75, label="TeleMetrum Altitude")

plt.xlim([-5,42])
#plt.ylim([-50,130])
#ax1.legend(loc=1)
plt.show()

## GPS

The GPS was not locked on the launch rail. Interestingly our COTS GPS and SDR GPS were also having a very difficult time tracking sats while on the rail. We appear to have very bad GPS reception near the (large and metal) launch rail.

The TeleMetrum reported lock and at least 5 stat for the whole flight once it got in the air.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"TeleMetrum Reported GPS State")
plt.ylabel(r"#")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(tele_time, tele_gpsnsat, alpha=0.75, label="Number of Sats")
plt.plot(tele_time, tele_gpslock, alpha=0.75, label="Lock State")


plt.xlim([-5,42])
plt.ylim([0,8])
ax1.legend(loc=1)
plt.show()

Much like the Venus 8 COTS GPS, the Telemetrum was unable to figure out that the rocket was in the air. It reported an altitude of the 0 meters for the entire flight, even after being on parachutes.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"TeleMetrum Reported GPS Altitude")
plt.ylabel(r"Altitude AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(tele_time, divide(subtract(tele_gpsalt, 1390), 1000.0), alpha=0.75, label="")


#plt.xlim([-5,42])
plt.ylim([0,6])
#ax1.legend(loc=1)
plt.show()

The GPS on the TeleMetrum reports the signal to noise ratio for each PRN in the GPS constellation. We can look to see if there is any pattern in the SNR's over time, or if any satellites that were below the horizon were reported as locked (they shouldn't be).

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"TeleMetrum Reported GPS SNR")
plt.ylabel(r"SNR")
plt.xlabel(r"Mission Elapsed Time [s]")

# Sats in the sky: 1,4,7,11,13,15,17,24,28,30
plt.plot(tele_time, sat1, alpha=0.75, label="1")
plt.plot(tele_time, sat4, alpha=0.75, label="4")
plt.plot(tele_time, sat7, alpha=0.75, label="7")
plt.plot(tele_time, sat11, alpha=0.75, label="11")
plt.plot(tele_time, sat13, alpha=0.75, label="13")
plt.plot(tele_time, sat15, alpha=0.75, label="15")
plt.plot(tele_time, sat17, alpha=0.75, label="17")
plt.plot(tele_time, sat24, alpha=0.75, label="24")
plt.plot(tele_time, sat28, alpha=0.75, label="28")
plt.plot(tele_time, sat30, alpha=0.75, label="30")

plt.xlim([-5,42])
#plt.ylim([0,6])
ax1.legend(loc=1, title="PRN")
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"TeleMetrum Reported GPS SNR, Sats Below Horizon (False Positives)")
plt.ylabel(r"SNR")
plt.xlabel(r"Mission Elapsed Time [s]")

# Sats in the sky: 1,4,7,11,13,15,17,24,28,30
plt.plot(tele_time, sat2, alpha=0.75, label="1")
plt.plot(tele_time, sat3, alpha=0.75, label="3")
plt.plot(tele_time, sat5, alpha=0.75, label="5")
plt.plot(tele_time, sat6, alpha=0.75, label="6")
plt.plot(tele_time, sat8, alpha=0.75, label="8")
plt.plot(tele_time, sat9, alpha=0.75, label="9")
plt.plot(tele_time, sat10, alpha=0.75, label="10")
plt.plot(tele_time, sat12, alpha=0.75, label="12")
plt.plot(tele_time, sat14, alpha=0.75, label="14")
plt.plot(tele_time, sat16, alpha=0.75, label="16")
plt.plot(tele_time, sat18, alpha=0.75, label="18")
plt.plot(tele_time, sat19, alpha=0.75, label="19")
plt.plot(tele_time, sat20, alpha=0.75, label="20")
plt.plot(tele_time, sat21, alpha=0.75, label="21")
plt.plot(tele_time, sat22, alpha=0.75, label="22")
plt.plot(tele_time, sat23, alpha=0.75, label="23")
plt.plot(tele_time, sat25, alpha=0.75, label="25")
plt.plot(tele_time, sat26, alpha=0.75, label="26")
plt.plot(tele_time, sat27, alpha=0.75, label="27")
plt.plot(tele_time, sat29, alpha=0.75, label="29")
plt.plot(tele_time, sat31, alpha=0.75, label="31")
plt.plot(tele_time, sat32, alpha=0.75, label="32")


plt.xlim([-5,42])
plt.ylim([0,40])
#ax1.legend(loc=1, title="PRN")
plt.show()